In [1]:
import numpy as np
import cv2
from collections import deque
import tensorflow as tf
import pandas as pd
import time

# Load your trained model
model = tf.keras.models.load_model('Model4.keras')

# Function for gamma correction
def gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Function to extract and preprocess images
def extract_and_preprocess_images(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    cropped_faces, original_images, left_eyes, right_eyes = [], [], [], []

    for (x, y, w, h) in faces:
        cropped_face = gray[y:y+h, x:x+w]
        cropped_faces.append(cropped_face)
        
        center_x, center_y = x + w // 2, y + h // 2
        margin_ratio = 0.25
        side_length = max(w, h)
        margin = int(side_length * margin_ratio)
        side_length_with_margin = side_length + 2 * margin
        
        crop_x1 = max(center_x - side_length_with_margin // 2, 0)
        crop_y1 = max(center_y - side_length_with_margin // 2, 0)
        crop_x2 = min(center_x + side_length_with_margin // 2, gray.shape[1])
        crop_y2 = min(center_y + side_length_with_margin // 2, gray.shape[0])
        
        original_cropped = gray[crop_y1:crop_y2, crop_x1:crop_x2]
        original_images.append(original_cropped)
        
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for i, (ex, ey, ew, eh) in enumerate(eyes):
            eye_roi = roi_gray[ey:ey+eh, ex:ex+ew]
            if i == 0:
                left_eyes.append(eye_roi)
            elif i == 1:
                right_eyes.append(eye_roi)

    if cropped_faces:
        cropped_faces = [cv2.resize(img, (227, 227)) for img in cropped_faces]
        cropped_faces = [gamma_correction(img) for img in cropped_faces]
        cropped_faces = [img / 255.0 for img in cropped_faces]
    else:
        cropped_faces = None

    if original_images:
        original_images = [cv2.resize(img, (227, 227)) for img in original_images]
        original_images = [gamma_correction(img) for img in original_images]
        original_images = [img / 255.0 for img in original_images]
    else:
        original_images = None

    if left_eyes:
        left_eyes = [cv2.resize(img, (30, 30)) for img in left_eyes]
        left_eyes = [gamma_correction(img) for img in left_eyes]
        left_eyes = [img / 255.0 for img in left_eyes]
    else:
        left_eyes = None

    if right_eyes:
        right_eyes = [cv2.resize(img, (30, 30)) for img in right_eyes]
        right_eyes = [gamma_correction(img) for img in right_eyes]
        right_eyes = [img / 255.0 for img in right_eyes]
    else:
        right_eyes = None

    return cropped_faces, original_images, left_eyes, right_eyes

# Initialize video capture
cap = cv2.VideoCapture(0)
x1, y1 = 150, 150  
x2, y2 = 550, 550
grid_points_original = cv2.imread('grid_points.jpg', 1)
radius = 20
color = (0, 255, 0)

# Deque to store the last 10 predictions
predictions = deque(maxlen=10)

# Initialize the Excel file
excel_file = 'Benchmark_Prabhat_9july.xlsx'
df = pd.DataFrame(columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
df.to_excel(excel_file, index=False)

# Record the start time
start_time = time.time()

while True:
    ret, frame = cap.read()
    frame = frame[y1:y2, x1:x2]
    
    if not ret:
        break

    cropped_faces, original_images, left_eyes, right_eyes = extract_and_preprocess_images(frame)

    if cropped_faces and original_images and left_eyes and right_eyes:
        points = model.predict([np.array(original_images), np.array(cropped_faces), np.array(left_eyes), np.array(right_eyes)])
        points = abs(np.round(points).astype(int))
        x, y = points[0][0], points[0][1]
        
        # Add the current prediction to the deque
        predictions.append((x, y))
        
        # Compute the average of the stored predictions
        avg_x = int(np.mean([p[0] for p in predictions]))
        avg_y = int(np.mean([p[1] for p in predictions]))
        
        # Create a copy of the original grid_points image
        grid_points = grid_points_original.copy()
        cv2.circle(grid_points, (avg_x, avg_y), radius, color, -1)
        cv2.imshow('Test Image with Circle', grid_points)

        # Log data to the Excel file
        elapsed_time = int((time.time() - start_time) * 1000)
        new_data = pd.DataFrame([[elapsed_time, x, y, avg_x, avg_y]], columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
        df = pd.concat([df, new_data], ignore_index=True)
        df.to_excel(excel_file, index=False)

    cv2.imshow('Video Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

In [3]:
import numpy as np
import cv2
from collections import deque
import tensorflow as tf
import pandas as pd
import time

# Load your trained model
model = tf.keras.models.load_model('Model4.keras')

# Function for gamma correction
def gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Function to extract and preprocess images
def extract_and_preprocess_images(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    cropped_faces, original_images, left_eyes, right_eyes = [], [], [], []

    for (x, y, w, h) in faces:
        cropped_face = gray[y:y+h, x:x+w]
        cropped_faces.append(cropped_face)
        
        center_x, center_y = x + w // 2, y + h // 2
        margin_ratio = 0.25
        side_length = max(w, h)
        margin = int(side_length * margin_ratio)
        side_length_with_margin = side_length + 2 * margin
        
        crop_x1 = max(center_x - side_length_with_margin // 2, 0)
        crop_y1 = max(center_y - side_length_with_margin // 2, 0)
        crop_x2 = min(center_x + side_length_with_margin // 2, gray.shape[1])
        crop_y2 = min(center_y + side_length_with_margin // 2, gray.shape[0])
        
        original_cropped = gray[crop_y1:crop_y2, crop_x1:crop_x2]
        original_images.append(original_cropped)
        
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for i, (ex, ey, ew, eh) in enumerate(eyes):
            eye_roi = roi_gray[ey:ey+eh, ex:ex+ew]
            if i == 0:
                left_eyes.append(eye_roi)
            elif i == 1:
                right_eyes.append(eye_roi)

    if cropped_faces:
        cropped_faces = [cv2.resize(img, (227, 227)) for img in cropped_faces]
        cropped_faces = [gamma_correction(img) for img in cropped_faces]
        cropped_faces = [img / 255.0 for img in cropped_faces]
    else:
        cropped_faces = None

    if original_images:
        original_images = [cv2.resize(img, (227, 227)) for img in original_images]
        original_images = [gamma_correction(img) for img in original_images]
        original_images = [img / 255.0 for img in original_images]
    else:
        original_images = None

    if left_eyes:
        left_eyes = [cv2.resize(img, (30, 30)) for img in left_eyes]
        left_eyes = [gamma_correction(img) for img in left_eyes]
        left_eyes = [img / 255.0 for img in left_eyes]
    else:
        left_eyes = None

    if right_eyes:
        right_eyes = [cv2.resize(img, (30, 30)) for img in right_eyes]
        right_eyes = [gamma_correction(img) for img in right_eyes]
        right_eyes = [img / 255.0 for img in right_eyes]
    else:
        right_eyes = None

    return cropped_faces, original_images, left_eyes, right_eyes

# Initialize video capture
cap = cv2.VideoCapture(0)
x1, y1 = 150, 150  
x2, y2 = 550, 550
grid_points_original = cv2.imread('grid_points.jpg', 1)
radius = 20
color = (0, 255, 0)

# Deque to store the last 10 predictions
predictions = deque(maxlen=10)

# Initialize the Excel file
excel_file = 'Benchmark_Prabhat_9july.xlsx'
df = pd.DataFrame(columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
df.to_excel(excel_file, index=False)

# Record the start time
start_time = time.time()

while True:
    ret, frame = cap.read()
    frame = frame[y1:y2, x1:x2]
    
    if not ret:
        break

    cropped_faces, original_images, left_eyes, right_eyes = extract_and_preprocess_images(frame)

    # Log the elapsed time
    elapsed_time = int((time.time() - start_time) * 1000)
    
    if cropped_faces and original_images and left_eyes and right_eyes:
        points = model.predict([np.array(original_images), np.array(cropped_faces), np.array(left_eyes), np.array(right_eyes)])
        points = abs(np.round(points).astype(int))
        x, y = points[0][0], points[0][1]
        
        # Add the current prediction to the deque
        predictions.append((x, y))
        
        # Compute the average of the stored predictions
        avg_x = int(np.mean([p[0] for p in predictions]))
        avg_y = int(np.mean([p[1] for p in predictions]))
        
        # Create a copy of the original grid_points image
        grid_points = grid_points_original.copy()
        cv2.circle(grid_points, (avg_x, avg_y), radius, color, -1)
        cv2.imshow('Test Image with Circle', grid_points)
        
        # Log the predictions
        new_data = pd.DataFrame([[elapsed_time, x, y, avg_x, avg_y]], columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
    else:
        # Log 'eye not detected' when eyes are not detected
        new_data = pd.DataFrame([[elapsed_time, 'eye not detected', 'eye not detected', 'eye not detected', 'eye not detected']], 
                                columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
    
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_excel(excel_file, index=False)

    cv2.imshow('Video Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━

### Additional Change

In [2]:
import numpy as np
import cv2
from collections import deque
import tensorflow as tf
import pandas as pd
import time

# Load your trained model
model = tf.keras.models.load_model('Model4.keras')

# Function for gamma correction
def gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Function to extract and preprocess images
def extract_and_preprocess_images(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    cropped_faces, original_images, left_eyes, right_eyes = [], [], [], []

    for (x, y, w, h) in faces:
        cropped_face = gray[y:y+h, x:x+w]
        cropped_faces.append(cropped_face)
        
        center_x, center_y = x + w // 2, y + h // 2
        margin_ratio = 0.25
        side_length = max(w, h)
        margin = int(side_length * margin_ratio)
        side_length_with_margin = side_length + 2 * margin
        
        crop_x1 = max(center_x - side_length_with_margin // 2, 0)
        crop_y1 = max(center_y - side_length_with_margin // 2, 0)
        crop_x2 = min(center_x + side_length_with_margin // 2, gray.shape[1])
        crop_y2 = min(center_y + side_length_with_margin // 2, gray.shape[0])
        
        original_cropped = gray[crop_y1:crop_y2, crop_x1:crop_x2]
        original_images.append(original_cropped)
        
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for i, (ex, ey, ew, eh) in enumerate(eyes):
            eye_roi = roi_gray[ey:ey+eh, ex:ex+ew]
            if i == 0:
                left_eyes.append(eye_roi)
            elif i == 1:
                right_eyes.append(eye_roi)

    if cropped_faces:
        cropped_faces = [cv2.resize(img, (227, 227)) for img in cropped_faces]
        cropped_faces = [gamma_correction(img) for img in cropped_faces]
        cropped_faces = [img / 255.0 for img in cropped_faces]
    else:
        cropped_faces = None

    if original_images:
        original_images = [cv2.resize(img, (227, 227)) for img in original_images]
        original_images = [gamma_correction(img) for img in original_images]
        original_images = [img / 255.0 for img in original_images]
    else:
        original_images = None

    if left_eyes:
        left_eyes = [cv2.resize(img, (30, 30)) for img in left_eyes]
        left_eyes = [gamma_correction(img) for img in left_eyes]
        left_eyes = [img / 255.0 for img in left_eyes]
    else:
        left_eyes = None

    if right_eyes:
        right_eyes = [cv2.resize(img, (30, 30)) for img in right_eyes]
        right_eyes = [gamma_correction(img) for img in right_eyes]
        right_eyes = [img / 255.0 for img in right_eyes]
    else:
        right_eyes = None

    return cropped_faces, original_images, left_eyes, right_eyes

# Initialize video capture
cap = cv2.VideoCapture(0)
x1, y1 = 150, 150  
x2, y2 = 550, 550
grid_points_original = cv2.imread('grid_points.jpg', 1)
radius = 20
color = (0, 255, 0)
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_color = (255, 255, 255)
thickness = 2

# Deque to store the last 10 predictions
predictions = deque(maxlen=10)

# Initialize the Excel file
excel_file = 'Benchmark_Prabhat_9july.xlsx'
df = pd.DataFrame(columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
df.to_excel(excel_file, index=False)

# Record the start time
start_time = time.time()

while True:
    ret, frame = cap.read()
    frame = frame[y1:y2, x1:x2]
    
    if not ret:
        break

    cropped_faces, original_images, left_eyes, right_eyes = extract_and_preprocess_images(frame)

    # Log the elapsed time
    elapsed_time = int((time.time() - start_time) * 1000)
    
    if cropped_faces and original_images and left_eyes and right_eyes:
        points = model.predict([np.array(original_images), np.array(cropped_faces), np.array(left_eyes), np.array(right_eyes)])
        points = abs(np.round(points).astype(int))
        x, y = points[0][0], points[0][1]
        
        # Add the current prediction to the deque
        predictions.append((x, y))
        
        # Compute the average of the stored predictions
        avg_x = int(np.mean([p[0] for p in predictions]))
        avg_y = int(np.mean([p[1] for p in predictions]))
        
        # Create a copy of the original grid_points image
        grid_points = grid_points_original.copy()
        cv2.circle(grid_points, (avg_x, avg_y), radius, color, -1)
        
        # Display predicted coordinates on the screen
        text = f'Prediction: ({x}, {y})'
        cv2.putText(grid_points, text, (frame.shape[1] - 250, frame.shape[0] - 20), font, font_scale, font_color, thickness)
        
        cv2.imshow('Test Image with Circle', grid_points)
        
        # Log the predictions
        new_data = pd.DataFrame([[elapsed_time, x, y, avg_x, avg_y]], columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
    else:
        # Log 'eye not detected' when eyes are not detected
        new_data = pd.DataFrame([[elapsed_time, 'eye not detected', 'eye not detected', 'eye not detected', 'eye not detected']], 
                                columns=['Time (ms)', 'X', 'Y', 'Avg_X', 'Avg_Y'])
    
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_excel(excel_file, index=False)

    cv2.imshow('Video Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━